In [1]:
import copy

import pandas
from scipy.io import arff
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from dl85 import DL85Predictor
from dl85 import DL85Classifier
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
import MyCode.utils as utils
import graphviz
from MyCode import relabeling
from MyCode.relabeling import Leaf

In [2]:
pd.set_option('max_columns', None)
FILE_NAME = "relab_Lawsuit"

# src: https://www.kaggle.com/hjmjerry/gender-discrimination
df = pandas.read_csv("dataset_perso/Lawsuit.csv")
del df['ID']
salary_mean = np.mean(np.array(df["Sal94"].tolist(), float))
for i in range(0,len(df)):
    if df.at[i, "Sal94"] >= salary_mean:
        df.at[i, "Salary_mean"] = 1
    else:
        df.at[i, "Salary_mean"] = 0
del df["Sal94"]
del df["Sal95"]
col = ['Rank', 'Dept']
df = df.drop(['Prate', 'Exper'], axis=1)
df = pd.get_dummies(df, columns=col)
for i in range(0,len(df)):
    if df.at[i, "Gender"] == 1:
        df.at[i, "Gender"] = 0
    else:
        df.at[i, "Gender"] = 1
X = df.loc[:, ~df.columns.isin(['Gender', 'Salary_mean'])]
y = df['Salary_mean']
sensitive = df['Gender']
print(type(sensitive))

<class 'pandas.core.series.Series'>


In [3]:

k = 0
depth = 5
random_state = 5
min_supp = 2

X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = utils.train_test_split(random_state, X, y, sensitive)


In [4]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=depth, random_state=0)
clf.fit(X_train, list(y_train))
y_pred_train = clf.predict(X_train)
accuracy_train = round(accuracy_score(y_train, y_pred_train),10)
print("Accuracy DL8.5 on training set =", accuracy_train)
discrimation_train_pred = round(utils.discrimination(y_pred_train, sensitive_train), 10)
print("Discrimination of classifier on train(pred)", discrimation_train_pred)

Accuracy DL8.5 on training set = 0.9186602871
Discrimination of classifier on train(pred) 0.3469368158


In [5]:
from MyCode import relabeling_fairlearn

leafs_relab = relabeling_fairlearn.leaves_to_relabel(clf, X_train, y_train, y_pred_train, sensitive_train, 0)
leafs_relab = sorted(leafs_relab, key=lambda x: x.ratio, reverse=True)
for leaf in leafs_relab:
    print(leaf)
    print()
    relabeling_fairlearn.browse_and_relab(clf, leaf.node_id)

Path: format -> (feature, type, node id)
((10, 'left', 0), (9, 'left', 1), (4, 'right', 2), (7, 'left', 8), (8, 'right', 9), (-2, 'leaf', 11)) 
node_id: 11 
The effect of relabeling the leaf on accuracy: 0.0
The effect of relabeling the leaf on discrimination: -0.004321106203188017 
ratio: 4.321106203188017e+35 
contingency table: 
[0.0, 0.004784688995215311]
[0.004784688995215311, 0.0]
transactions: [94, 102]

Path: format -> (feature, type, node id)
((10, 'left', 0), (9, 'right', 1), (2, 'right', 15), (1, 'right', 21), (0, 'right', 25), (-2, 'leaf', 27)) 
node_id: 27 
The effect of relabeling the leaf on accuracy: -0.004784688995215308
The effect of relabeling the leaf on discrimination: -0.05108507777991164 
ratio: 10.67678125600154 
contingency table: 
[0.028708133971291867, 0.019138755980861243]
[0.019138755980861243, 0.023923444976076555]
transactions: [114, 128, 129, 131, 132, 133, 134, 137, 141, 154, 155, 157, 159, 160, 161, 162, 165, 166, 173]

Path: format -> (feature, type, 

In [6]:
y_pred_train = clf.predict(X_train)
accuracy_train = round(accuracy_score(y_train, y_pred_train),10)
print("Accuracy DL8.5 on training set =", accuracy_train)
discrimation_train_pred = round(utils.discrimination(y_pred_train, sensitive_train), 10)
print("Discrimination of classifier on train(pred)", discrimation_train_pred)

Accuracy DL8.5 on training set = 0.6076555024
Discrimination of classifier on train(pred) -0.1240637603
